In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from matplotlib.image import imread
import numpy as np
from PIL import Image
import cv2 
import os 
import glob 
from numpy.linalg import eig
from numpy import array
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from numpy import linalg as LA


**Bonus**

In [ ]:
images = []
for filename in sorted(os.listdir("/content/drive/MyDrive/ORL")):
    img = cv2.imread(os.path.join("/content/drive/MyDrive/ORL",filename),cv2.IMREAD_GRAYSCALE)
    if img is not None:
       ar=img.flatten()
     
       images.append(ar)
#images  
data =np.array(images)
data

array([[ 34,  34,  33, ...,  37,  40,  33],
       [ 42,  41,  44, ...,  42,  43,  41],
       [ 48,  49,  45, ...,  47,  46,  46],
       ...,
       [113, 112, 111, ...,  87,  87,  89],
       [112, 109, 116, ...,  93,  88,  92],
       [111, 114, 112, ...,  88,  86,  92]], dtype=uint8)

In [ ]:
vec=np.empty([400,1])
count=0;
for i in range(0,400):
  if (i%10==0):
    count+=1
  vec[i]=count
  
DataLab=np.hstack((data,vec)) 
DataLab.shape


(400, 10305)

**Use different Training and Test splits. Change the number of
instances per subject to be 7 and keep 3 instances per subject for testing.**

In [ ]:
test=[]
train=[]
trainVec=np.empty([160,1])
testVec=np.empty([120,1])
k=0
l=0
flag=0
counter1=7
counter2=0
for i in range(1,400):
  if (counter1>0 and flag ==0 ):
    if (i%2==0):
      test.append(data[i])
      testVec[l]=DataLab[i,10304]
      l+=1
    else :
      train.append(data[i]) 
      trainVec[k]=DataLab[i,10304]
      k+=1
    counter1-=1
    if (counter1==0):
      flag=1
    counter2=3
    continue
  if (counter2 > 0):
  
    
    counter2-=1
    if (counter2==0):
      flag=0
    counter1=7 
train=np.array(train)
test=np.array(test)
print(train.shape)
print(test.shape)
print(trainVec.shape)
print(testVec.shape)

(160, 10304)
(120, 10304)
(160, 1)
(120, 1)


In [ ]:
mean=[]
temp=[]
count=0
temp.append(train[0])
for i in range(1,160):
  if (i%4==0):
    mean.append(np.mean(temp,0))
    temp.clear()
    count+=1
  temp.append(train[i])
 
mean.append(np.mean(temp,0))

mean=np.array(mean)
print(mean.shape)
mean


(40, 10304)


array([[ 52.  ,  51.  ,  45.5 , ...,  68.25,  60.  ,  67.  ],
       [136.5 , 135.  , 135.5 , ...,  20.25,  21.75,  31.5 ],
       [ 49.5 ,  47.75,  46.25, ...,  30.75,  30.5 ,  34.75],
       ...,
       [ 55.5 ,  54.  ,  55.  , ...,  64.5 ,  62.5 ,  61.25],
       [122.  , 120.  , 122.25, ...,  62.  ,  62.25,  60.  ],
       [109.25, 111.  , 110.75, ...,  84.  ,  83.5 ,  81.  ]])

In [ ]:
overallMean=np.mean(train,0)

Sb=np.zeros((10304,10304))

temp=np.empty([10304,1])
for i in range(0,40):
  
  temp=np.subtract(mean[i].reshape(10304,1),overallMean.reshape(10304,1))
  temp=4*temp
  Sb=np.add(Sb,np.dot(temp,np.transpose(temp)))
 

Sb


array([[ 749103.6  ,  749524.4  ,  739583.4  , ..., -110906.8  ,
         -84074.8  ,  -61167.2  ],
       [ 749524.4  ,  752546.975,  741335.225, ..., -105461.2  ,
         -80509.95 ,  -57374.3  ],
       [ 739583.4  ,  741335.225,  732708.975, ..., -113118.2  ,
         -86204.45 ,  -65252.3  ],
       ...,
       [-110906.8  , -105461.2  , -113118.2  , ..., 1145148.4  ,
        1051155.4  , 1017728.6  ],
       [ -84074.8  ,  -80509.95 ,  -86204.45 , ..., 1051155.4  ,
         994803.9  ,  970391.6  ],
       [ -61167.2  ,  -57374.3  ,  -65252.3  , ..., 1017728.6  ,
         970391.6  ,  977766.4  ]])

In [ ]:
#computing S total=s1+s2+s3+...
temp=[]
temp.append(train[0])
empty=[]
k=0
St=np.zeros((10304,10304))
Sk=[]
centered=[]
for i in range(1,160):
  if (i%4==0):
    centered=temp-np.transpose(mean[k])
    
    temp.clear()
    k+=1
    Sk=np.dot(np.transpose(centered),centered)
    centered=np.array(empty)
   
    
    St=np.add(St,Sk)
  temp.append(train[i])
centered=temp-np.transpose(mean[k])
 
temp.clear()
k+=1
Sk=np.dot(np.transpose(centered),centered)
centered=np.array(empty)
St=np.add(St,Sk)
St

array([[18664.5 , 17874.5 , 17934.75, ...,  1586.5 ,  -400.5 , -1731.  ],
       [17874.5 , 19410.25, 18666.25, ...,   824.75, -2017.75, -2338.  ],
       [17934.75, 18666.25, 20335.25, ...,  1293.5 ,  -826.75, -1593.75],
       ...,
       [ 1586.5 ,   824.75,  1293.5 , ..., 71872.5 , 56728.25, 51422.25],
       [ -400.5 , -2017.75,  -826.75, ..., 56728.25, 74410.  , 65149.25],
       [-1731.  , -2338.  , -1593.75, ..., 51422.25, 65149.25, 78901.5 ]])

In [ ]:
stInv=LA.inv(St)
arr=np.dot(stInv,Sb)


In [ ]:
eigen_values, eigen_vectors =LA.eigh(arr)
w=np.real( np.delete(eigen_vectors, np.s_[0:10265], axis=1)   )
print(w)
trainRed=np.zeros((160,39))
testRed=np.zeros((120,39))
trainRed=np.dot(train,w)
testRed=np.dot(test,w)

[[ 0.00239231 -0.00927234  0.00746086 ... -0.00296065  0.00176506
  -0.00194198]
 [ 0.00298689 -0.01148162  0.00994731 ... -0.00334391  0.00200062
  -0.00220199]
 [ 0.00241488 -0.00942156  0.00742204 ... -0.00287737  0.00171182
  -0.00187888]
 ...
 [-0.00563782 -0.00029528 -0.00695126 ... -0.00165264 -0.0015103
  -0.00242767]
 [ 0.00875538  0.00445979  0.00499052 ...  0.00396219  0.00348312
   0.00380923]
 [-0.00016156 -0.00325369  0.0013149  ... -0.00148604 -0.00086312
  -0.00124438]]


In [ ]:
classifier = KNeighborsClassifier(n_neighbors=1)
classifier.fit(trainRed, trainVec.ravel())  
test_predict = classifier.predict(testRed)   
true_predicted_count=0
for j in range(0,120):
   if test_predict[j] ==testVec[j]:
      true_predicted_count+=1

true_predicted_count
accuracy=true_predicted_count/120
accuracy=accuracy*100
accuracy

91.66666666666666

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(trainRed)

train_pca3 = scaler.transform(trainRed)
test_pca3 = scaler.transform(testRed)

from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=1)
classifier.fit(trainRed, trainVec)

y_pred = classifier.predict(testRed)

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(testVec, y_pred))
print(classification_report(testVec, y_pred))

[[1 0 0 ... 0 0 0]
 [0 2 0 ... 0 1 0]
 [0 0 3 ... 0 0 0]
 ...
 [0 0 0 ... 2 0 0]
 [0 0 0 ... 0 3 0]
 [0 0 0 ... 0 0 3]]
              precision    recall  f1-score   support

         1.0       1.00      0.33      0.50         3
         2.0       1.00      0.67      0.80         3
         3.0       1.00      1.00      1.00         3
         4.0       1.00      1.00      1.00         3
         5.0       1.00      1.00      1.00         3
         6.0       1.00      1.00      1.00         3
         7.0       0.75      1.00      0.86         3
         8.0       0.75      1.00      0.86         3
         9.0       1.00      1.00      1.00         3
        10.0       0.75      1.00      0.86         3
        11.0       1.00      0.67      0.80         3
        12.0       0.75      1.00      0.86         3
        13.0       1.00      1.00      1.00         3
        14.0       0.75      1.00      0.86         3
        15.0       1.00      1.00      1.00         3
        16.0   

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.
